In [ ]:
import siibra
from nilearn import plotting
from IPython.display import display, Markdown

### Define two points in MNI space

We start by specifying some points in the reference space. Here, we use the MNI space. We can find such point, amongst other possibilities, by clicking them in the atlas viewer.

In [ ]:
# These are copy-pasted from the interactive atlas viewer:
c1 = "-25.650mm, -2.750mm, -33.750mm"
c2 = "-37.350mm, -81.050mm, -6.300mm"

In [ ]:
P1 = siibra.arrays.parse_coordinate_str(c1)
P2 = siibra.arrays.parse_coordinate_str(c2)
plotting.view_markers([P1,P2], ['red', 'cyan'], marker_size=10)  

### Assign brain regions to the 3D points

We assign the points to brain regions from the Julich-Brain cytoarchitectonic atlas, using a certain location tolerance of 5mm standard deviation.

In [ ]:
atlas = siibra.atlases['human']
atlas.select(parcellation="julich-brain 2.5")
regions1,regions2 = atlas.assign_coordinates("mni152",[c1,c2],sigma_mm=5)

Let's look at the most probable area found for each point.

In [ ]:
region1,statmap1,value1 = regions1[0]
region2,statmap2,value2 = regions2[0]
view1 = plotting.plot_stat_map(statmap1,title=f"{region1.name} ({value1*100:.1f}%)",cut_coords=P1)
view2 = plotting.plot_stat_map(statmap2,title=f"{region2.name} ({value2*100:.1f}%)",cut_coords=P2)
view1.add_markers([P1],['cyan'])
view2.add_markers([P2],['cyan'])

### Look for white matter bundles connecting the regions

Next, we look at the most probable region associated to each point, considering them as a source and target region to investigate connectivity. In order to find white matter fibre bundles which are likely to connect them, we use the probability maps of the white matter fibre bundle parcellation, and assign both the source and target region from Julich-Brain to the fibre bundles. The intersection of the resulting bundles gives us those that are likely to provide connections from source to target.

In [ ]:
# get the probabilistic maps of long fibre bundles
atlas.select(parcellation='long bundle')
bundlemaps = atlas.selected_parcellation.get_map("mni152","continuous")

# assign the source and target region to the bundles, 
# using their probability maps found above.
bundles1 = bundlemaps.assign(statmap1,msg=f"Find bundles touching {region1.name}")
bundles2 = bundlemaps.assign(statmap2,msg=f"Find bundles touching {region2.name}")
                
# intersect the two sets of related fiber bundles
intersection = {
    b[0]:b[1] for b in bundles1+bundles2
    if  any(b1[0]==b[0] for b1 in bundles1) 
    and any(b2[0]==b[0] for b2 in bundles2) }

# plot each of them bundles in the intersection
for bundle,bundlemap in intersection.items():
    view=plotting.plot_stat_map(bundlemap,
                                title=f"{region1.name} and {region2.name} connected by {bundle.name} ")
    view.add_overlay(statmap1)
    view.add_overlay(statmap2)
    view.add_markers([P1,P2],['white'])
        

We might want to inspect the scene in 3D using a more advanced viewer like napari.

In [ ]:
import napari
with napari.gui_qt():
    
    # show the brain template
    tpl = atlas.get_template("mni152").fetch()
    viewer = napari.view_image(tpl.dataobj,affine=tpl.affine,name="MNI152",
        ndisplay=3,rgb=False,colormap='gray',rendering='average',opacity=.5)
        
    # show the first region
    viewer.add_image(statmap1.dataobj,affine=statmap1.affine,name=region1.name,
        rgb=False, rendering='mip',colormap='blue',blending='additive',gamma=.4)

    # show the second region
    viewer.add_image(statmap2.dataobj,affine=statmap2.affine,name=region2.name,
        rgb=False, rendering='mip',colormap='red',blending='additive',gamma=.4)
    
    # show the bundle
    viewer.add_image(bundlemap.dataobj,affine=bundlemap.affine,name=bundle.name,
        rgb=False, rendering='mip',colormap='turbo',blending='additive',gamma=.4,contrast_limits=(30,100))
    
    # show the original 3D points
    viewer.add_points(data=[P1,P2],size=3)

### Find connectivity in terms of streamlines from DTI

We also investigate the connectivtiy of the two regions as measured by in-vivo imaging. To do so, we select the first region in the atlas, and search for connectivity profiles.

In [ ]:
# select the source region
atlas.select(parcellation='julich-brain 2.5',region=region1)
with siibra.QUIET:
    profiles = atlas.get_features(siibra.modalities.ConnectivityProfile)
print(f"Siibra found {len(profiles)} different connectivity profiles going out from {region1.name}:")

There are in general multiple features, so let's see which types of connectivity profiles have been found?

In [ ]:
for i,p in enumerate(profiles):
    print(f"{i:3} - {p.src_name}")

We choose a structural connectivity profile, and decode the names of the connected regions.

In [ ]:
p = profiles[0]
print(p.src_name)
display(Markdown(p.src_info))
with siibra.QUIET:
    connections = p.decode(atlas.selected_parcellation)

Create a bar plot of the connectivity profile, and identify the target region from above.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

i = [r for v,r in connections].index(region2)

N = max(30,i)
X = range(N)
Y = [v for v,_ in connections[:N]]
L = [r.name for v,r in connections[:N]]

fig  = plt.figure(figsize=(12,4))
ax1  = fig.add_subplot()
ax1.set_xticks(X)
ax1.set_xticklabels(L,rotation=60,ha='right')
ax1.bar(X,Y,0.8)

# where is region2?
i = [r for v,r in connections].index(region2)
ax1.bar(i,connections[i][0],0.9,color='r')

fig.gca().set_title(f"Connection strengths from area {region1.name}")
plt.show()